# Clase 10 - Sistemas de ecuaciones lineales III

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import linalg
from scipy.sparse import diags

In [2]:
plt.style.use('seaborn-poster')

#formato de impresión de los números en los arreglos de Numpy
#np.set_printoptions(formatter={'float': '{: .4f}'.format})

### Método de Gauss-Seidel

El método de Gauss-Seidel con relajación está dado por

\begin{equation*}
x_{i} \leftarrow \frac{\omega}{A_{i i}}\left(b_{i}-\sum_{j=1 \atop j \neq i}^{n} A_{i j} x_{j}\right)+(1-\omega) x_{i}, \quad i=1,2, \ldots, n
\end{equation*}

Esta ecuación es usada para calcular cada elemento de $\vec{x}$ usando siempre el último valor disponible de $x_i$.

Para calcular el factor de relajación, usamos la solución con $\omega=0$ para las primeras $k\ge 5$ iteraciones para calcular el valor óptimo

\begin{equation*}
\omega_{\mathrm{opt}} \approx \frac{2}{1+\sqrt{1-\left(\Delta x^{(k+p)} / \Delta x^{(k)}\right)^{1 / p}}}
\end{equation*}

donde $p$ es un entero positivo.

In [3]:
def Ecuaciones(x,A,b,omega):
    n = len(x)
    
    for i in range(0,n):
        suma = 0.0
        for j in range(0,n):
            if j != i:
                suma = suma + A[i][j]*x[j]
        x[i] = (1.0 - omega)*x[i] + omega*(b[i] - suma)/A[i][i]
    
    return (x)

In [4]:
def GaussSeidel(Ecuaciones,x,A,b,tol=1.0e-9):
    omega = 1.0
    k = 10
    p = 1
    xold = np.zeros(len(x))
    
    diag = False
    D = np.diag(np.abs(A))
    S = np.sum(np.abs(A), axis=1) - D
    if np.all(D > S):
        diag = True
        print ('La matriz A es diagonalmente dominada')
    
    for i in range(1,1001):
        xold = x.copy()
        x = Ecuaciones(x,A,b,omega)
        dx = np.sqrt(np.dot(x-xold,x-xold))
        if dx < tol:
            return (x,i,omega)
        if i == k: 
            dx1 = dx
        if i == k + p:
            dx2 = dx
            omega = 2.0/(1.0 + np.sqrt(1.0 - (dx2/dx1)**(1.0/p)))
    
    print ('Método de Gauss-Seidel no converge')

Resolvamos el sistema de ecuaciones 

\begin{equation*}
\left[\begin{array}{rrr}4 & -1 & 1 \\ -1 & 4 & -2 \\ 1 & -2 & 4\end{array}\right]\left[\begin{array}{l}x_{1} \\ x_{2} \\ x_{3}\end{array}\right]=\left[\begin{array}{r}12 \\ -1 \\ 5\end{array}\right]
\end{equation*}


In [5]:
A = np.array([[4,-1,1],[-1,4,-2],[1,-2,4]],float)
b = np.array([12,-1,5],float)

# valor inicial de x
x = np.array([1,1,1],float)

In [6]:
x,i,omega = GaussSeidel(Ecuaciones,x,A,b)
print ('La solución es =',x)
print ('Número de iteraciones =',i)
print ('Factor de relajación =',omega)

La matriz A es diagonalmente dominada
La solución es = [3. 1. 1.]
Número de iteraciones = 2
Factor de relajación = 1.0


In [7]:
print (np.dot(A,x)-b)

[0. 0. 0.]


Ahora resolvamos el siguiente sistema de ecuaciones 

\begin{equation*}
\left[\begin{array}{rrrrrrrrr}2 & -1 & 0 & 0 & \ldots & 0 & 0 & 0 & 1 \\ -1 & 2 & -1 & 0 & \ldots & 0 & 0 & 0 & 0 \\ 0 & -1 & 2 & -1 & \ldots & 0 & 0 & 0 & 0 \\ \vdots & \vdots & \vdots & \vdots & & \vdots & \vdots & \vdots & \vdots \\ 0 & 0 & 0 & 0 & \ldots & -1 & 2 & -1 & 0 \\ 0 & 0 & 0 & 0 & \ldots & 0 & -1 & 2 & -1 \\ 1 & 0 & 0 & 0 & \ldots & 0 & 0 & -1 & 2\end{array}\right]\left[\begin{array}{c}x_{1} \\ x_{2} \\ x_{3} \\ \vdots \\ x_{n-2} \\ x_{n-1} \\ x_{n}\end{array}\right]=\left[\begin{array}{c}0 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0 \\ 1\end{array}\right]
\end{equation*}

cuya solución está dada por $x_i=-n/4+i/2$ con $i=1,2,\ldots,n$.

In [8]:
n = 20
A = diags([-1, 2, -1], [-1, 0, 1], shape=(n, n),dtype=float).toarray()
A[0][n-1] = 1
A[n-1][0] = 1
b = np.zeros(n)
b[n-1] = 1

In [9]:
x = np.zeros(n)
x,i,omega = GaussSeidel(Ecuaciones,x,A,b)
print ('La solución es =',x)
print ('Número de iteraciones =',i)
print ('Factor de relajación =',omega)

La solución es = [-4.50000000e+00 -4.00000000e+00 -3.50000000e+00 -3.00000000e+00
 -2.50000000e+00 -2.00000000e+00 -1.50000000e+00 -9.99999997e-01
 -4.99999998e-01  2.14046747e-09  5.00000002e-01  1.00000000e+00
  1.50000000e+00  2.00000000e+00  2.50000000e+00  3.00000000e+00
  3.50000000e+00  4.00000000e+00  4.50000000e+00  5.00000000e+00]
Número de iteraciones = 259
Factor de relajación = 1.7054523107131407


In [10]:
print (np.dot(A,x)-b)

[ 3.16644488e-11  5.89737148e-11  6.43085585e-11  6.79483136e-11
  6.98121561e-11  6.98685554e-11  6.81299461e-11  6.46572795e-11
  5.95542504e-11  5.29650768e-11  4.50693877e-11  3.60844687e-11
  2.62498911e-11  1.58193458e-11  5.07593967e-12 -5.71098724e-12
 -1.62603264e-11 -2.63034039e-11 -3.55879770e-11 -1.68762782e-11]


Ahora resolvamos el sistema

\begin{equation*}
\left[\begin{array}{rrrrrrrrr}4 & -1 & 0 & 0 & \cdots & 0 & 0 & 0 & 1 \\ -1 & 4 & -1 & 0 & \cdots & 0 & 0 & 0 & 0 \\ 0 & -1 & 4 & -1 & \cdots & 0 & 0 & 0 & 0 \\ \vdots & \vdots & \vdots & \vdots & \cdots & \vdots & \vdots & \vdots & \vdots \\ 0 & 0 & 0 & 0 & \cdots & -1 & 4 & -1 & 0 \\ 0 & 0 & 0 & 0 & \cdots & 0 & -1 & 4 & -1 \\ 1 & 0 & 0 & 0 & \cdots & 0 & 0 & -1 & 4\end{array}\right]\left[\begin{array}{c}x_{1} \\ x_{2} \\ x_{3} \\ \vdots \\ x_{n-2} \\ x_{n-1} \\ x_{n}\end{array}\right]=\left[\begin{array}{c}0 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0 \\ 100\end{array}\right]
\end{equation*}

In [11]:
n = 20
A = diags([-1, 4, -1], [-1, 0, 1], shape=(n, n),dtype=float).toarray()
A[0][n-1] = 1
A[n-1][0] = 1
b = np.zeros(n)
b[n-1] = 100

In [12]:
x = np.zeros(n)
x,i,omega = GaussSeidel(Ecuaciones,x,A,b)
print ('La solución es =',x)
print ('Número de iteraciones =',i)
print ('Factor de relajación =',omega)

La matriz A es diagonalmente dominada
La solución es = [-7.73502692e+00 -2.07259421e+00 -5.55349941e-01 -1.48805549e-01
 -3.98722562e-02 -1.06834753e-02 -2.86164518e-03 -7.63105381e-04
 -1.90776345e-04  8.65000649e-14  1.90776346e-04  7.63105381e-04
  2.86164518e-03  1.06834753e-02  3.98722562e-02  1.48805549e-01
  5.55349941e-01  2.07259421e+00  7.73502692e+00  2.88675135e+01]
Número de iteraciones = 21
Factor de relajación = 1.0976797558334


In [13]:
print (np.dot(A,x)-b)

[-1.48084212e-10 -7.79927234e-11 -3.97273325e-11 -1.47250614e-11
 -3.10559911e-12  6.21991608e-13  1.09880507e-12  7.23615124e-13
  3.48203020e-13 -3.39806688e-13  1.97763894e-12 -5.66361628e-13
 -1.67102226e-13 -1.55639390e-14  5.28743715e-15  3.33066907e-15
  1.11022302e-15  1.58442148e-11  2.93631786e-11 -1.24629196e-11]
